#### Data loading

In [1]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, FunctionTransformer, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer

In [2]:
df_reg = pd.read_csv("merged_data_2018.csv")

/var/folders/wk/w4dq25x94cg_26_p83q6tnlh0000gn/T/ipykernel_12561/2502907041.py:1: DtypeWarning: Columns (9,31,33,35,46) have mixed types. Specify dtype option on import or set low_memory=False.
  df_reg = pd.read_csv("merged_data_2018.csv")


In [3]:
df_reg.head()

,bldgclass,landuse,proxcode,bsmtcode,assessland,assesstot,exempttot,yearalter1,yearalter2,landmark,...,days_pending,market_heat,new_construction_sale,sales_count,nyur,redfunds,GDP,median_income,cpi,bldgclass_letter
0,A4,1.0,3.0,1.0,16680.0,77760.0,0.0,1989.0,0.0,NaN,...,53.0,61.0,415.0,16754.0,3.9,2.56,26272.011,78920.0,7.99264,A
1,C1,2.0,3.0,0.0,297000.0,1264500.0,0.0,2011.0,0.0,NaN,...,50.0,66.0,517.0,20756.0,3.8,2.33,26272.011,78920.0,7.99264,C
2,C1,2.0,3.0,0.0,297000.0,1264500.0,0.0,2011.0,0.0,NaN,...,50.0,66.0,517.0,20756.0,3.8,2.33,26272.011,78920.0,7.99264,C
3,C4,2.0,3.0,2.0,150750.0,815400.0,815400.0,2009.0,0.0,NaN,...,53.0,61.0,415.0,16754.0,3.9,2.56,26272.011,78920.0,7.99264,C
4,C7,4.0,3.0,2.0,122850.0,2337120.0,0.0,0.0,2009.0,NaN,...,50.0,66.0,517.0,20756.0,3.8,2.33,26272.011,78920.0,7.99264,C


##### tutaj sprawdzam na wplyw outlierow na sale price

In [4]:
from scipy import stats

col = 'sale price'

Q1 = df_reg[col].quantile(0.25)
Q3 = df_reg[col].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df_clean = df_reg[(df_reg[col] >= lower_bound) & (df_reg[col] <= upper_bound)]

In [5]:
df_clean["sale price"].describe()

count    2.378320e+05
mean     6.812092e+05
std      3.936725e+05
min      1.000000e+00
25%      3.940000e+05
50%      6.350000e+05
75%      9.000000e+05
max      1.867500e+06
Name: sale price, dtype: float64

In [6]:
df_reg["sale price"].describe()

count    2.618670e+05
mean     1.206654e+06
std      5.611312e+06
min      1.000000e+00
25%      4.200000e+05
50%      6.800000e+05
75%      9.990000e+05
max      8.696129e+08
Name: sale price, dtype: float64

#### jezeli to zrobimy to dzialamy na datasecie bez outlierow - jesli nie to nie odpalaj ponizszej komorki

In [7]:
# optional
df_reg = df_clean

In [8]:
df_reg["sale_date"] = pd.to_datetime(df_reg["sale_date"])

In [9]:
df_reg['sale_year'] = df_reg['sale_date'].dt.year
df_reg['sale_month'] = df_reg['sale_date'].dt.month
df_reg['sale_day'] = df_reg['sale_date'].dt.day
df_reg['sale_weekday'] = df_reg['sale_date'].dt.weekday  # Monday=0, Sunday=6
df_reg['sale_quarter'] = df_reg['sale_date'].dt.quarter

#### Wybor zmiennych

In [11]:
to_drop = [
    "bldgclass", "building class category", "building class at present", 
    "sale date", "Date", "month_end", "ease-ment", "apartment number", 
    "block", "lot", 'tax class at present', 'building class at time of sale', 
    "sale_date"]
numeric_features = [
    # Continuous/Count variables
    'assessland', 'assesstot', 'exempttot', 'builtfar', 'residfar', 'commfar', 'facilfar',
    'land square feet', 'gross square feet', 'zhvi', 'zori', 'zordi',
    'inventory', 'days_pending', 'market_heat', 'sales_count', 'GDP', 'median_income', 'cpi',
    # Additional numeric (if applicable)
    'easements',

    'residential units', 'commercial units', 'total units',
    'year built',

    "sale_year", "sale_month", "sale_day", "sale_weekday", "sale_quarter",
    # jak robisz autogluon to odkomentuj sale price
    "sale price" 
]

# Boolean (Flag) variables - often numeric (0/1 or many zeros/missing) in nature
boolean_features = [
    'firm07_flag', 'pfirm15_flag', 'changed_building_class',
    'new_listing', 'new_pendly_listing', 'new_construction_sale',
    'yearalter1', 'yearalter2', 'ltdheight',
    'zonedist1', 'zonedist2', 'overlay1', 'overlay2',
    'spdist1', 'spdist2','landmark', 'ownertype',
]

# Object (Categorical) variables
categorical_features = [
    'firecomp', 'sanitsub', 'neighborhood',
    'bldgclass_letter',
    # Identifier/Code variables (numeric but likely categorical)
    'landuse', 'proxcode', 'bsmtcode', 'lottype', 'cd', 'schooldist', 'council',
    'policeprct', 'healthcenterdistrict', 'healtharea', 'sanitboro', 'sanitdistrict',
    ' zip code',  # note: zip code is numeric but typically categorical
    'tax class at time of sale', 'buidling_class_num',
]

In [12]:
df_reg = df_reg.drop(columns=to_drop)

In [13]:
df_reg.columns

Index(['landuse', 'proxcode', 'bsmtcode', 'assessland', 'assesstot',
       'exempttot', 'yearalter1', 'yearalter2', 'landmark', 'builtfar',
       'residfar', 'commfar', 'facilfar', 'lottype', 'condono', 'cd',
       'firm07_flag', 'pfirm15_flag', 'schooldist', 'council', 'firecomp',
       'policeprct', 'healthcenterdistrict', 'healtharea', 'sanitboro',
       'sanitdistrict', 'sanitsub', 'zonedist1', 'zonedist2', 'overlay1',
       'overlay2', 'spdist1', 'spdist2', 'spdist3', 'ltdheight', 'easements',
       'ownertype', 'borough', 'neighborhood', ' zip code',
       'residential units', 'commercial units', 'total units',
       'land square feet', 'gross square feet', 'year built',
       'tax class at time of sale', 'sale price', 'buidling_class_num',
       'changed_building_class', 'zhvi', 'zori', 'zordi', 'inventory',
       'new_listing', 'new_pendly_listing', 'days_pending', 'market_heat',
       'new_construction_sale', 'sales_count', 'nyur', 'redfunds', 'GDP',
       'media

#### Transformacje

Do zmiennych numerycznych

Klasa ktora obcina outliery poza 1 i 99 percentylem

In [57]:
class OutlierClipper(BaseEstimator, TransformerMixin):
    """
    Clips values in each column to lie between the lower and upper quantile thresholds.
    """
    def __init__(self, lower_quantile=0.01, upper_quantile=0.99):
        self.lower_quantile = lower_quantile
        self.upper_quantile = upper_quantile

    def fit(self, X, y=None):
        # Compute the lower and upper bounds for each column
        self.lower_bounds_ = np.nanpercentile(X, self.lower_quantile * 100, axis=0)
        self.upper_bounds_ = np.nanpercentile(X, self.upper_quantile * 100, axis=0)
        return self

    def transform(self, X):
        # Clip values for each column based on computed bounds
        return np.clip(X, self.lower_bounds_, self.upper_bounds_)


In [58]:
# Optional: Define a log transformation (using log1p to handle zeros safely)
# to pomaga z mocno skrzywionymi rozkladami
log_transformer = FunctionTransformer(np.log1p, validate=True)

# Build a numeric pipeline that imputes, clips outliers, applies log transform, and scales.
numeric_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),  # Fill missing values
    ('outlier_clipper', OutlierClipper(lower_quantile=0.01, upper_quantile=0.99)),
    ('log_transform', log_transformer),             # Apply log transform (if distributions are skewed)
    ('scaler', StandardScaler())                    # Standardize features
])

Zmienne 0-1

In [59]:
def to_binary(x):
    # 0 if the value is null, 0, "0", or an empty string; otherwise 1.
    def convert(val):
        if pd.isnull(val) or val == 0 or val == '0' or val == '':
            return 0
        else:
            return 1
    # Vectorize the conversion to apply elementwise
    vectorized_convert = np.vectorize(convert)
    return vectorized_convert(x)

boolean_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value=0)),  # Fill missing values with 0
    ('to_bool', FunctionTransformer(to_binary, validate=False))     # Convert to boolean (0/1)
])

Zmienne kategoryczne

Klasa do wartosci kategorycznych - jak jest ich mniej niz 20 000 (ok. 10% zbioru) to da other

In [27]:
class RareCategoryGrouper(BaseEstimator, TransformerMixin):
    """
    Groups categories that appear less than min_frequency times as 'other'.
    This transformer now accepts both pandas DataFrames and numpy arrays.
    """
    def __init__(self, min_frequency=10):
        self.min_frequency = min_frequency

    def fit(self, X, y=None):
        # Convert to DataFrame if X is not one
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)
        self.frequent_categories_ = {}
        for col in X.columns:
            counts = X[col].value_counts()
            self.frequent_categories_[col] = counts[counts >= self.min_frequency].index.tolist()
        return self

    def transform(self, X):
        # Convert to DataFrame if needed
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)
        X_transformed = X.copy()
        for col in X_transformed.columns:
            # Use get to provide an empty list if column wasn't seen during fit.
            frequent = self.frequent_categories_.get(col, [])
            X_transformed[col] = X_transformed[col].apply(lambda x: x if x in frequent else "other")
        return X_transformed

In [28]:
categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('rare_grouper', RareCategoryGrouper(5000)),  # Adjust threshold as needed
    ('to_string', FunctionTransformer(lambda x: x.astype(str))),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

In [29]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_pipeline, numeric_features),
        ('bool', boolean_pipeline, boolean_features),
        ('cat', categorical_pipeline, categorical_features),
    ]
)

#### Sprawdzenie dzialania transformatorow

tutaj czy rzadkie kategorie sie zapisuja

In [17]:
sample_array = np.array([['A', 'X'], ['B', 'Y'], ['A', 'Z'], ['C', 'X'], ['D', 'X']])
sample_df = pd.DataFrame(sample_array, columns=['col1', 'col2'])
sample_df

,col1,col2
0,A,X
1,B,Y
2,A,Z
3,C,X
4,D,X


In [18]:
rcg = RareCategoryGrouper(min_frequency=2)
print(rcg.fit_transform(sample_df.values))

       0      1
0      A      X
1  other  other
2      A  other
3  other      X
4  other      X


In [19]:
numeric_sample = pd.DataFrame({
    'num1': [100, 200, np.nan, 100000000, 50],
    'num2': [10, 15, 0, 100, np.nan]
})
numeric_sample

,num1,num2
0,100.0,10.0
1,200.0,15.0
2,NaN,0.0
3,100000000.0,100.0
4,50.0,NaN


tutaj czy dziala clip dla outlierow (clip != usuniecie wartosci tylko przeksztalcenie ich)

In [20]:
OutlierClipper().fit_transform(numeric_sample)

,num1,num2
0,100.0,10.00
1,200.0,15.00
2,NaN,0.30
3,97000006.0,97.45
4,51.5,NaN


In [21]:
numeric_pipeline.fit_transform(numeric_sample)

array([[-0.51865539, -0.1039363 ],
       [-0.3930663 ,  0.17382146],
       [-0.44526398, -1.6320555 ],
       [ 1.993318  ,  1.51429391],
       [-0.63633233,  0.04787643]])

tutaj jak dzialaj kolumny 0-1

In [22]:
bool_sample = pd.DataFrame({
    'bool_col': [np.nan, 0, '0', '', 'LH-1', 'OtherValue', 5, '']
})
bool_sample

,bool_col
0,NaN
1,0
2,0
3,
4,LH-1
5,OtherValue
6,5
7,


In [23]:
boolean_pipeline.fit_transform(bool_sample)

array([[0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0]])

Kategorie:

In [24]:
sample_cat = pd.DataFrame({
    'firecomp': ['F1', 'F1', 'F2', 'F3', 'F4', 'F5', 'F1', 'F6', 'F6', 'F6', 'F7'],
    'sanitsub': ['S1', 'S2', 'S2', 'S1', 'S1', 'S3', 'S3', 'S3', 'S4', np.nan, 'S4'],
    'neighborhood': ['NB1', 'NB1', 'NB2', 'NB3', 'NB1', 'NB2', 'NB2', 'NB3', 'NB3', 'NB3', 'NB3'],
    'bldgclass_letter': ['A', 'B', 'A', 'C', 'C', 'C', 'A', np.nan, 'B', 'A', 'A'],
    'landuse': ['1', '2', '3', '2', '1', '1', '2', '3', '4', '4', '4']
})
sample_cat

,firecomp,sanitsub,neighborhood,bldgclass_letter,landuse
0,F1,S1,NB1,A,1
1,F1,S2,NB1,B,2
2,F2,S2,NB2,A,3
3,F3,S1,NB3,C,2
4,F4,S1,NB1,C,1
5,F5,S3,NB2,C,1
6,F1,S3,NB2,A,2
7,F6,S3,NB3,NaN,3
8,F6,S4,NB3,B,4
9,F6,NaN,NB3,A,4


In [25]:
categorical_pipeline2 = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('rare_grouper', RareCategoryGrouper(3)),  # Adjust threshold as needed
    ('to_string', FunctionTransformer(lambda x: x.astype(str))),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

In [26]:
categorical_pipeline2.fit_transform(sample_cat).toarray()

array([[1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1.],
       [0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0.],
       [0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0.]])

In [27]:
ohe = categorical_pipeline2.named_steps['onehot']

# Retrieve the feature names generated by the encoder
feature_names = ohe.get_feature_names_out()
print(feature_names)

['x0_F1' 'x0_F6' 'x0_other' 'x1_S1' 'x1_S3' 'x1_other' 'x2_NB1' 'x2_NB2'
 'x2_NB3' 'x3_A' 'x3_C' 'x3_other' 'x4_1' 'x4_2' 'x4_4' 'x4_other']


#### Modele

##### LGBM

In [37]:
import lightgbm as lgb
from sklearn.metrics import r2_score, root_mean_squared_error

X = df_reg.drop("sale price", axis=1)
y = df_reg["sale price"]

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# First fit the preprocessor on the training data
X_train_transformed = preprocessor.fit_transform(X_train)
# Then transform validation and test sets
X_val_transformed = preprocessor.transform(X_val)
X_test_transformed = preprocessor.transform(X_test)

# Print shapes to confirm transformation was successful
print(f"X_train shape: {X_train_transformed.shape}")
print(f"X_val shape: {X_val_transformed.shape}")
print(f"X_test shape: {X_test_transformed.shape}")

train_data = lgb.Dataset(X_train_transformed, label=y_train)
val_data = lgb.Dataset(X_val_transformed, label=y_val, reference=train_data)

X_train shape: (166482, 177)
X_val shape: (35675, 177)
X_test shape: (35675, 177)


In [41]:
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'num_leaves': 31,           # controls complexity; adjust based on your data
    'learning_rate': 0.05,      # a smaller learning rate can improve accuracy but requires more rounds
    'feature_fraction': 0.9,    # randomly select part of features on each iteration for robustness
    'bagging_fraction': 0.8,    # randomly select part of data on each iteration for better generalization
    'bagging_freq': 5,          # frequency for bagging; set to 0 to disable
    'verbose': 0
}

In [42]:
model = lgb.train(
    params,
    train_data,
    num_boost_round=20000,
    valid_sets=[train_data, val_data],
    valid_names=['train', 'validation'],
    callbacks=[
        lgb.early_stopping(400), 
        lgb.log_evaluation(100)]
)

Training until validation scores don't improve for 400 rounds
[100]	train's rmse: 272312	validation's rmse: 275208
[200]	train's rmse: 263757	validation's rmse: 269118
[300]	train's rmse: 258541	validation's rmse: 266196
[400]	train's rmse: 254052	validation's rmse: 263748
[500]	train's rmse: 250081	validation's rmse: 262043
[600]	train's rmse: 246786	validation's rmse: 260639
[700]	train's rmse: 243383	validation's rmse: 259203
[800]	train's rmse: 240490	validation's rmse: 258210
[900]	train's rmse: 237761	validation's rmse: 257203
[1000]	train's rmse: 235234	validation's rmse: 256391
[1100]	train's rmse: 232607	validation's rmse: 255482
[1200]	train's rmse: 230222	validation's rmse: 254783
[1300]	train's rmse: 228153	validation's rmse: 254237
[1400]	train's rmse: 225915	validation's rmse: 253685
[1500]	train's rmse: 223646	validation's rmse: 252954
[1600]	train's rmse: 221655	validation's rmse: 252371
[1700]	train's rmse: 219552	validation's rmse: 251845
[1800]	train's rmse: 217635	v

In [43]:
y_pred = model.predict(X_val_transformed, num_iteration=model.best_iteration)
rmse = root_mean_squared_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)
print(rmse, r2)

238795.75683755052 0.6346433411552408


In [44]:
y_pred_test = model.predict(X_test_transformed, num_iteration=model.best_iteration)
r2_test = r2_score(y_test, y_pred_test)
r2_test

0.6266782964053643

##### AUTOGLUON

In [15]:
from autogluon.tabular import TabularPredictor

/Users/patryk/venv-torch/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
X = df_reg

X_train, X_temp= train_test_split(X, test_size=0.3, random_state=42)
X_val, X_test= train_test_split(X_temp, test_size=0.5, random_state=42)

trening bez outlierow

In [ ]:
predictor_custom = TabularPredictor(label="sale price", problem_type="regression", eval_metric="root_mean_squared_error", verbosity=3).fit(X_train, time_limit=960)

No path specified. Models will be saved in: "AutogluonModels/ag-20250308_144906"
Verbosity: 3 (Detailed Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.9.6
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 24.3.0: Thu Jan  2 20:24:24 PST 2025; root:xnu-11215.81.4~3/RELEASE_ARM64_T6030
CPU Count:          11
GPU Count:          0
Memory Avail:       6.59 GB / 18.00 GB (36.6%)
Disk Space Avail:   32.78 GB / 460.43 GB (7.1%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='experimental' : New in v1.2: Pre-trained foundation model + parallel fits. The absolute best accuracy without consideration for inference speed. Does not support GPU.
	presets='best'         

[50]	valid_set's rmse: 283422
[100]	valid_set's rmse: 274477
[150]	valid_set's rmse: 271301
[200]	valid_set's rmse: 269475
[250]	valid_set's rmse: 268338
[300]	valid_set's rmse: 267381
[350]	valid_set's rmse: 266478
[400]	valid_set's rmse: 265725
[450]	valid_set's rmse: 264999
[500]	valid_set's rmse: 264401
[550]	valid_set's rmse: 263826
[600]	valid_set's rmse: 263442
[650]	valid_set's rmse: 262884
[700]	valid_set's rmse: 262617
[750]	valid_set's rmse: 262340
[800]	valid_set's rmse: 262207
[850]	valid_set's rmse: 261635
[900]	valid_set's rmse: 261419
[950]	valid_set's rmse: 261273
[1000]	valid_set's rmse: 261059
[1050]	valid_set's rmse: 260725
[1100]	valid_set's rmse: 260518
[1150]	valid_set's rmse: 260435
[1200]	valid_set's rmse: 260304
[1250]	valid_set's rmse: 260139
[1300]	valid_set's rmse: 260042
[1350]	valid_set's rmse: 259956
[1400]	valid_set's rmse: 259744
[1450]	valid_set's rmse: 259580
[1500]	valid_set's rmse: 259417
[1550]	valid_set's rmse: 259195
[1600]	valid_set's rmse: 259

Saving /Users/patryk/test/kon/AutogluonModels/ag-20250308_144906/models/LightGBMXT/model.pkl
Saving /Users/patryk/test/kon/AutogluonModels/ag-20250308_144906/utils/attr/LightGBMXT/y_pred_proba_val.pkl
	-253617.6293	 = Validation score   (-root_mean_squared_error)
	62.81s	 = Training   runtime
	0.2s	 = Validation runtime
	12555.9	 = Inference  throughput (rows/s | 2500 batch size)
Saving /Users/patryk/test/kon/AutogluonModels/ag-20250308_144906/models/trainer.pkl
Fitting model: LightGBM ... Training model for up to 895.00s of the 894.99s of remaining time.
	Fitting LightGBM with 'num_gpus': 0, 'num_cpus': 11
	Fitting 10000 rounds... Hyperparameters: {'learning_rate': 0.05}


[50]	valid_set's rmse: 280794
[100]	valid_set's rmse: 272510
[150]	valid_set's rmse: 269430
[200]	valid_set's rmse: 267562
[250]	valid_set's rmse: 265861
[300]	valid_set's rmse: 264487
[350]	valid_set's rmse: 263354
[400]	valid_set's rmse: 262774
[450]	valid_set's rmse: 262123
[500]	valid_set's rmse: 261647
[550]	valid_set's rmse: 261066
[600]	valid_set's rmse: 260559
[650]	valid_set's rmse: 260106
[700]	valid_set's rmse: 259498
[750]	valid_set's rmse: 258893
[800]	valid_set's rmse: 258699
[850]	valid_set's rmse: 258263
[900]	valid_set's rmse: 258341
[950]	valid_set's rmse: 257937
[1000]	valid_set's rmse: 257421
[1050]	valid_set's rmse: 257102
[1100]	valid_set's rmse: 256746
[1150]	valid_set's rmse: 256474
[1200]	valid_set's rmse: 256251
[1250]	valid_set's rmse: 256107
[1300]	valid_set's rmse: 255652
[1350]	valid_set's rmse: 255324
[1400]	valid_set's rmse: 255091
[1450]	valid_set's rmse: 254967
[1500]	valid_set's rmse: 254907
[1550]	valid_set's rmse: 254651
[1600]	valid_set's rmse: 254

Saving /Users/patryk/test/kon/AutogluonModels/ag-20250308_144906/models/LightGBM/model.pkl
Saving /Users/patryk/test/kon/AutogluonModels/ag-20250308_144906/utils/attr/LightGBM/y_pred_proba_val.pkl
	-247997.1531	 = Validation score   (-root_mean_squared_error)
	59.89s	 = Training   runtime
	0.14s	 = Validation runtime
	17607.7	 = Inference  throughput (rows/s | 2500 batch size)
Saving /Users/patryk/test/kon/AutogluonModels/ag-20250308_144906/models/trainer.pkl
Fitting model: RandomForestMSE ... Training model for up to 834.81s of the 834.81s of remaining time.
	Fitting RandomForestMSE with 'num_gpus': 0, 'num_cpus': 11
Saving /Users/patryk/test/kon/AutogluonModels/ag-20250308_144906/models/RandomForestMSE/model.pkl
Saving /Users/patryk/test/kon/AutogluonModels/ag-20250308_144906/utils/attr/RandomForestMSE/y_pred_proba_val.pkl
	-239098.6527	 = Validation score   (-root_mean_squared_error)
	96.29s	 = Training   runtime
	0.06s	 = Validation runtime
	44805.4	 = Inference  throughput (rows/s

0:	learn: 386646.3057467	test: 390387.5728132	best: 390387.5728132 (0)	total: 113ms	remaining: 18m 49s
20:	learn: 313087.6091011	test: 318005.5371723	best: 318005.5371723 (20)	total: 912ms	remaining: 7m 13s
40:	learn: 292482.8235324	test: 297767.2112169	best: 297767.2112169 (40)	total: 1.54s	remaining: 6m 13s
60:	learn: 285036.3405085	test: 291078.0768987	best: 291078.0768987 (60)	total: 2.34s	remaining: 6m 20s
80:	learn: 281597.3151635	test: 287965.1528745	best: 287965.1528745 (80)	total: 3.26s	remaining: 6m 39s
100:	learn: 279006.4401101	test: 285604.5170167	best: 285604.5170167 (100)	total: 4.08s	remaining: 6m 40s
120:	learn: 277042.0591235	test: 284029.8757703	best: 284029.8757703 (120)	total: 4.88s	remaining: 6m 38s
140:	learn: 275503.3527634	test: 282634.0167921	best: 282634.0167921 (140)	total: 5.68s	remaining: 6m 37s
160:	learn: 274118.9310948	test: 281528.9934838	best: 281528.9934838 (160)	total: 6.49s	remaining: 6m 36s
180:	learn: 272925.5126673	test: 280465.4805776	best: 280

Saving /Users/patryk/test/kon/AutogluonModels/ag-20250308_144906/models/CatBoost/model.pkl
Saving /Users/patryk/test/kon/AutogluonModels/ag-20250308_144906/utils/attr/CatBoost/y_pred_proba_val.pkl
	-255598.7507	 = Validation score   (-root_mean_squared_error)
	436.92s	 = Training   runtime
	0.02s	 = Validation runtime
	138805.2	 = Inference  throughput (rows/s | 2500 batch size)
Saving /Users/patryk/test/kon/AutogluonModels/ag-20250308_144906/models/trainer.pkl
Fitting model: ExtraTreesMSE ... Training model for up to 301.25s of the 301.24s of remaining time.
	Fitting ExtraTreesMSE with 'num_gpus': 0, 'num_cpus': 11
Saving /Users/patryk/test/kon/AutogluonModels/ag-20250308_144906/models/ExtraTreesMSE/model.pkl
Saving /Users/patryk/test/kon/AutogluonModels/ag-20250308_144906/utils/attr/ExtraTreesMSE/y_pred_proba_val.pkl
	-242295.1297	 = Validation score   (-root_mean_squared_error)
	44.51s	 = Training   runtime
	0.04s	 = Validation runtime
	61908.9	 = Inference  throughput (rows/s | 250

wyniki na zbiorze ze wszystkim

In [ ]:
predictor_custom.evaluate(X_test)

Loading: /Users/patryk/test/kon/AutogluonModels/ag-20250308_140219/models/ExtraTreesMSE/model.pkl
Loading: /Users/patryk/test/kon/AutogluonModels/ag-20250308_140219/models/KNeighborsDist/model.pkl
Loading: /Users/patryk/test/kon/AutogluonModels/ag-20250308_140219/models/LightGBM/model.pkl
Loading: /Users/patryk/test/kon/AutogluonModels/ag-20250308_140219/models/NeuralNetFastAI/model.pkl
Loading: /Users/patryk/test/kon/AutogluonModels/ag-20250308_140219/models/NeuralNetFastAI/model-internals.pkl
Loading: /Users/patryk/test/kon/AutogluonModels/ag-20250308_140219/models/NeuralNetTorch/model.pkl
Loading: /Users/patryk/test/kon/AutogluonModels/ag-20250308_140219/models/WeightedEnsemble_L2/model.pkl


{'root_mean_squared_error': -2916395.4546280648,
 'mean_squared_error': -8505362447775.236,
 'mean_absolute_error': -427163.4225973594,
 'r2': 0.7440674775120014,
 'pearsonr': 0.9038150964796793,
 'median_absolute_error': -152291.875}

wyniki bez outlierow

In [17]:
tp = TabularPredictor.load("AutogluonModels/ag-20250308_144906/")

In [18]:
tp.evaluate(X_test)

{'root_mean_squared_error': -235288.54280152995,
 'mean_squared_error': -55360698373.6674,
 'mean_absolute_error': -154785.18588567272,
 'r2': 0.6408188619843449,
 'pearsonr': 0.8010439466664497,
 'median_absolute_error': -96012.75}